In [2]:
from __future__ import absolute_import, division, print_function
import codecs
import glob
import logging
import multiprocessing
import os
import re
import string
import nltk
import gensim.models.word2vec as w2v
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import gensim
from nltk.corpus import stopwords

In [3]:
stopwords = set(stopwords.words('spanish'))

In [4]:
hhgroups = pd.read_csv('rap.csv')
hhgroups.head()
letras = list(hhgroups['letra'])
letra_limpio = []

Elinando líneas vacías y procesamiento de las letras de las canciones

In [6]:
for i in range(len(letras)):
    cancion_limpia = []
    for linea in letras[i].split("\n"):
        if ("[" not in linea and "(" not in linea and linea != ""):
            # pasamos linea a minusculas y eliminamos la puntuación
            linea = bytes(linea,'utf-8').decode('utf-8','ignore')
            linea = "".join(c for c in linea if (c not in string.punctuation and c not in ['','¡','¿'])).lower()
            linea = linea.split(" ")
            for palabra in list(linea):
                if palabra in stopwords or palabra in string.punctuation:
                    linea.remove(palabra)
            cancion_limpia += linea
    letra_limpio += [cancion_limpia]
    
print(letra_limpio[10][:10])
print(len(letra_limpio))

['nunca', 'pensé', 'desvíe', 'tracé', 'vía', 'merezco', 'muerte', 'vida', 'ayúdame', 'encontrar']
18650


sg 1 cbow 0

In [9]:
hhgroups2vec = w2v.Word2Vec(letra_limpio, sg = 1, seed = 1, workers = multiprocessing.cpu_count(), min_count = 50, window = 12)
hhgroups2vec.wv.most_similar("familia")
hhgroups2vec.wv.most_similar("españa")
hhgroups2vec.wv.most_similar("1")
hhgroups2vec.wv.most_similar("arte")
hhgroups2vec.wv.most_similar("música")
hhgroups2vec.wv.most_similar("joder")
hhgroups2vec.wv.most_similar("bien")

[('mal', 0.7205325961112976),
 ('cien', 0.6055631637573242),
 ('sé', 0.5994840860366821),
 ('así', 0.5817930102348328),
 ('pues', 0.5713661909103394),
 ('sien', 0.5619829893112183),
 ('preocupes', 0.5579007267951965),
 ('si', 0.5506299734115601),
 ('edén', 0.5410284996032715),
 ('rehén', 0.5403343439102173)]

In [10]:
def nearest_similarity_cosmul(start1,end1,end2):
    similarities = hhgroups2vec.wv.most_similar_cosmul(
        positive = [end2,start1],
        negative = [end1]
        )
    start2 = similarities[0][0] ##predicción
    print("{0} es a {1},  lo que {2} es a {3}".format(start1, end1, start2, end2))

In [11]:
nearest_similarity_cosmul("ser","siendo","haciendo")      
nearest_similarity_cosmul("amar","amor","odio") 
nearest_similarity_cosmul("rey","reina","princesa") 
nearest_similarity_cosmul("europa","españa","argentina") 

ser es a siendo,  lo que hacer es a haciendo
amar es a amor,  lo que odiar es a odio
rey es a reina,  lo que leo es a princesa
europa es a españa,  lo que ruinas es a argentina


Descartar las palabras fuera de lugar

In [12]:
hhgroups2vec.wv.doesnt_match("felicidad amor alegría envidia".split(" "))


'envidia'

Visualización con t-sne

Visualizando el modelo en 2D con T-SNE

In [18]:
def tsne_plot(model):
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity = 40, n_components = 2, init = 'pca', n_iter = 2500)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize = (64, 64)) 
    for i in range(len(x)):
        plt.scatter(x[i], y[i])
        plt.annotate(labels[i],
                     xy = (x[i], y[i]),
                     xytext = (5, 2),
                     textcoords ='offset points',
                     ha = 'right',
                     va = 'bottom')
    plt.show()

In [19]:
tsne_plot(hhgroups2vec)

AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4